# FILTER OF MAFS FROM UKB 50K
### (gVCFs FILTERED BY CH COORDINATES)


STEPS:

1. Open MAF file

2. Select only rows with real variants from MAFs

3. Rearrange info from last column

4. Eliminate "<NON_REF>" from ALT column

5. Eliminate variants with AD=None, DP=None, or DP=0

6. Split variants with >1 ALT & Calculate VAF

7. Annotate variant type

8. Reorder columns and create new column for VEP output

In [1]:
import pandas as pd
import os
import copy
pd.set_option('display.max_columns', 50)
#from tqdm import tqdm
#tqdm.pandas()

In [2]:
### 0. List MAF files (variants after filtering by CH coordinates)

# List vcf files after CH filtering in respectives directores
vcf_ch_files = os.listdir('/workspace/datasets/ukbiobank_ch/vcf_ch_50k')
vcf_ch_files = [file for file in vcf_ch_files if '.maf' in file]

In [3]:
### 1. Open MAF file

## PROCESS
# Open file (3rd file as an example)
maf = pd.read_csv('/workspace/datasets/ukbiobank_ch/vcf_ch_50k/' + vcf_ch_files[0], sep= '\t')


## CHECK
# Number of lines
print("Number of variants with <NON_REF>: " + str(len(maf)))

# Show some variants
maf.iloc[35:45,]

Number of variants with <NON_REF>: 13341


,#CHROM,POS,POS2,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,UKB_5336357_0230743959
35,1,1804382,1804382,.,C,<NON_REF>,.,.,END=1804405,GT:DP:GQ:MIN_DP:PL,"0/0:15:21:9:0,21,315"
36,1,1804406,1804406,.,C,<NON_REF>,.,.,END=1804601,GT:DP:GQ:MIN_DP:PL,"0/0:38:63:23:0,63,769"
37,1,1804602,1804602,.,G,<NON_REF>,.,.,END=1804681,GT:DP:GQ:MIN_DP:PL,"0/0:18:21:9:0,21,315"
38,1,1806375,1806375,.,T,<NON_REF>,.,.,END=1806441,GT:DP:GQ:MIN_DP:PL,"0/0:17:42:15:0,42,630"
39,1,1806442,1806442,.,G,<NON_REF>,.,.,END=1806455,GT:DP:GQ:MIN_DP:PL,"0/0:27:72:25:0,72,1007"
40,1,1806456,1806456,rs77354509,G,"A,<NON_REF>",514.77,.,BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...,GT:AD:DP:GQ:PGT:PID:PL:SB,"0/1:13,14,0:27:99:0|1:1806456_G_A:543,0,1150,5..."
41,1,1806457,1806457,.,G,<NON_REF>,.,.,END=1806460,GT:DP:GQ:MIN_DP:PL,"0/0:29:81:29:0,81,1215"
42,1,1806461,1806461,rs17363334,C,"T,<NON_REF>",546.77,.,BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...,GT:AD:DP:GQ:PGT:PID:PL:SB,"0/1:15,15,0:30:99:0|1:1806456_G_A:575,0,1147,6..."
43,1,1806462,1806462,.,C,<NON_REF>,.,.,END=1806564,GT:DP:GQ:MIN_DP:PL,"0/0:48:63:25:0,63,945"
44,1,1806565,1806565,.,A,<NON_REF>,.,.,END=1806638,GT:DP:GQ:MIN_DP:PL,"0/0:17:30:11:0,30,450"


In [4]:
### 2. Select only rows with real variants from MAFs

## PROCESS
# Select only variants NOT having "<NON_REF>" in "ALT"
maf_var = maf[~maf["ALT"].str.startswith("<NON_REF>")]

## CHECK
# Number of variants
print("Number of variants: " + str(len(maf_var)))

# Show first variants
maf_var.head()

Number of variants: 414


,#CHROM,POS,POS2,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,UKB_5336357_0230743959
40,1,1806456,1806456,rs77354509,G,"A,<NON_REF>",514.77,.,BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...,GT:AD:DP:GQ:PGT:PID:PL:SB,"0/1:13,14,0:27:99:0|1:1806456_G_A:543,0,1150,5..."
42,1,1806461,1806461,rs17363334,C,"T,<NON_REF>",546.77,.,BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...,GT:AD:DP:GQ:PGT:PID:PL:SB,"0/1:15,15,0:30:99:0|1:1806456_G_A:575,0,1147,6..."
88,1,43339569,43339569,rs16830693,A,"G,<NON_REF>",881.77,.,BaseQRankSum=1.575;DB;DP=41;ExcessHet=3.0103;M...,GT:AD:DP:GQ:PL:SB,"0/1:14,27,0:41:99:910,0,459,952,541,1493:5,9,6,21"
121,1,43346404,43346404,rs1760670,G,"A,<NON_REF>",232.77,.,BaseQRankSum=-3.638;DB;DP=19;ExcessHet=3.0103;...,GT:AD:DP:GQ:PL:SB,"0/1:11,8,0:19:99:261,0,437,294,461,755:11,0,8,0"
131,1,43349193,43349193,rs839995,T,"C,<NON_REF>",190.77,.,BaseQRankSum=3.403;DB;DP=17;ExcessHet=3.0103;M...,GT:AD:DP:GQ:PL:SB,"0/1:11,6,0:17:99:219,0,341,252,360,611:9,2,5,1"


In [19]:
### 3. Rearrange info from last column

## PROCESS
# Split column and add column names
split_ukbcol = maf_var[maf_var.columns[-1]].str.split(":",expand=True)
split_ukbcol.columns = ["GT","AD","DP","GQ","PGT","PID","PL","SB"]

# Reorder columns 4-7
for i in range(split_ukbcol.shape[0]):
    if split_ukbcol.iloc[i, 6] is None:
        split_ukbcol.iloc[i, 6] = split_ukbcol.iloc[i, 4]
        split_ukbcol.iloc[i, 4] = None
    if split_ukbcol.iloc[i, 7] is None:
        split_ukbcol.iloc[i, 7] = split_ukbcol.iloc[i, 5]
        split_ukbcol.iloc[i, 5] = None

# Select columns from original maf and merge with splitted columns
maf_var_spl = pd.concat([maf_var.iloc[:,[0,1,3,4,5,6,7]],
                         split_ukbcol, 
                         maf_var.iloc[:,8]], axis=1)
## CHECK
maf_var_spl

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,GT,AD,DP,GQ,PGT,PID,PL,SB,INFO
40,1,1806456,rs77354509,G,A,514.77,.,0/1,"13,14,0",27,99,0|1,1806456_G_A,"543,0,1150,588,1192,1780","12,1,14,0",BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...
42,1,1806461,rs17363334,C,T,546.77,.,0/1,"15,15,0",30,99,0|1,1806456_G_A,"575,0,1147,620,1192,1812","12,3,14,1",BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...
88,1,43339569,rs16830693,A,G,881.77,.,0/1,"14,27,0",41,99,None,None,"910,0,459,952,541,1493","5,9,6,21",BaseQRankSum=1.575;DB;DP=41;ExcessHet=3.0103;M...
121,1,43346404,rs1760670,G,A,232.77,.,0/1,"11,8,0",19,99,None,None,"261,0,437,294,461,755","11,0,8,0",BaseQRankSum=-3.638;DB;DP=19;ExcessHet=3.0103;...
131,1,43349193,rs839995,T,C,190.77,.,0/1,"11,6,0",17,99,None,None,"219,0,341,252,360,611","9,2,5,1",BaseQRankSum=3.403;DB;DP=17;ExcessHet=3.0103;M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13049,X,124066463,rs34397445,AT,A,358.73,.,0/1,"8,10,0",18,99,0|1,124066457_GT_G,"396,0,501,420,531,951","2,6,0,10",BaseQRankSum=0.318;DB;DP=18;ExcessHet=3.0103;M...
13062,X,124068468,rs113697794,C,CTATA,76.73,.,0/1,"4,3,0",7,99,None,None,"114,0,237,126,246,372","4,0,3,0",BaseQRankSum=-1.204;DB;DP=7;ExcessHet=3.0103;M...
13079,X,124071128,rs759815507,CT,C,3.39,.,0/1,"15,4,4,0",23,20,None,None,"38,0,363,20,261,381,97,362,381,466","13,2,8,0",BaseQRankSum=1.184;DB;DP=32;ExcessHet=3.0103;M...
13292,X,124392181,rs2076165,T,C,426.77,.,0/1,"28,15,0",43,99,None,None,"455,0,826,539,871,1410","13,15,7,8",BaseQRankSum=2.186;DB;DP=43;ExcessHet=3.0103;M...


In [20]:
### 4. Eliminate "<NON_REF>" from ALT column

## PROCESS
# Define function
def eliminate_nonref(alt):
    """"
    Eliminates "<NON_REF>" from ALT
    Initially checks that "<NON_REF>" is present
    """
    if alt.split(",")[-1] == "<NON_REF>":
        wo_nonref = ",".join(alt.split(",")[:-1])
        return wo_nonref
    else:
        return alt
# Eliminate "<NON_REF>"
maf_var_spl["ALT"] = maf_var_spl["ALT"].apply(lambda x: eliminate_nonref(x))

## CHECK
maf_var_spl

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,GT,AD,DP,GQ,PGT,PID,PL,SB,INFO
40,1,1806456,rs77354509,G,A,514.77,.,0/1,"13,14,0",27,99,0|1,1806456_G_A,"543,0,1150,588,1192,1780","12,1,14,0",BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...
42,1,1806461,rs17363334,C,T,546.77,.,0/1,"15,15,0",30,99,0|1,1806456_G_A,"575,0,1147,620,1192,1812","12,3,14,1",BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...
88,1,43339569,rs16830693,A,G,881.77,.,0/1,"14,27,0",41,99,None,None,"910,0,459,952,541,1493","5,9,6,21",BaseQRankSum=1.575;DB;DP=41;ExcessHet=3.0103;M...
121,1,43346404,rs1760670,G,A,232.77,.,0/1,"11,8,0",19,99,None,None,"261,0,437,294,461,755","11,0,8,0",BaseQRankSum=-3.638;DB;DP=19;ExcessHet=3.0103;...
131,1,43349193,rs839995,T,C,190.77,.,0/1,"11,6,0",17,99,None,None,"219,0,341,252,360,611","9,2,5,1",BaseQRankSum=3.403;DB;DP=17;ExcessHet=3.0103;M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13049,X,124066463,rs34397445,AT,A,358.73,.,0/1,"8,10,0",18,99,0|1,124066457_GT_G,"396,0,501,420,531,951","2,6,0,10",BaseQRankSum=0.318;DB;DP=18;ExcessHet=3.0103;M...
13062,X,124068468,rs113697794,C,CTATA,76.73,.,0/1,"4,3,0",7,99,None,None,"114,0,237,126,246,372","4,0,3,0",BaseQRankSum=-1.204;DB;DP=7;ExcessHet=3.0103;M...
13079,X,124071128,rs759815507,CT,C,3.39,.,0/1,"15,4,4,0",23,20,None,None,"38,0,363,20,261,381,97,362,381,466","13,2,8,0",BaseQRankSum=1.184;DB;DP=32;ExcessHet=3.0103;M...
13292,X,124392181,rs2076165,T,C,426.77,.,0/1,"28,15,0",43,99,None,None,"455,0,826,539,871,1410","13,15,7,8",BaseQRankSum=2.186;DB;DP=43;ExcessHet=3.0103;M...


In [21]:
### 5. Eliminate variants with AD=None, DP=None, or DP=0
maf_var_spl = maf_var_spl[~((maf_var_spl['AD'].isnull()) |
                            (maf_var_spl['DP'].isnull()) |
                            (maf_var_spl['AD'].str.split(',', expand=True)[0].isnull()) |
                            (maf_var_spl['AD'].str.split(',', expand=True)[1].isnull()) |
                            (maf_var_spl['DP']=="0"))]

maf_var_spl

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,GT,AD,DP,GQ,PGT,PID,PL,SB,INFO
40,1,1806456,rs77354509,G,A,514.77,.,0/1,"13,14,0",27,99,0|1,1806456_G_A,"543,0,1150,588,1192,1780","12,1,14,0",BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...
42,1,1806461,rs17363334,C,T,546.77,.,0/1,"15,15,0",30,99,0|1,1806456_G_A,"575,0,1147,620,1192,1812","12,3,14,1",BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...
88,1,43339569,rs16830693,A,G,881.77,.,0/1,"14,27,0",41,99,None,None,"910,0,459,952,541,1493","5,9,6,21",BaseQRankSum=1.575;DB;DP=41;ExcessHet=3.0103;M...
121,1,43346404,rs1760670,G,A,232.77,.,0/1,"11,8,0",19,99,None,None,"261,0,437,294,461,755","11,0,8,0",BaseQRankSum=-3.638;DB;DP=19;ExcessHet=3.0103;...
131,1,43349193,rs839995,T,C,190.77,.,0/1,"11,6,0",17,99,None,None,"219,0,341,252,360,611","9,2,5,1",BaseQRankSum=3.403;DB;DP=17;ExcessHet=3.0103;M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13049,X,124066463,rs34397445,AT,A,358.73,.,0/1,"8,10,0",18,99,0|1,124066457_GT_G,"396,0,501,420,531,951","2,6,0,10",BaseQRankSum=0.318;DB;DP=18;ExcessHet=3.0103;M...
13062,X,124068468,rs113697794,C,CTATA,76.73,.,0/1,"4,3,0",7,99,None,None,"114,0,237,126,246,372","4,0,3,0",BaseQRankSum=-1.204;DB;DP=7;ExcessHet=3.0103;M...
13079,X,124071128,rs759815507,CT,C,3.39,.,0/1,"15,4,4,0",23,20,None,None,"38,0,363,20,261,381,97,362,381,466","13,2,8,0",BaseQRankSum=1.184;DB;DP=32;ExcessHet=3.0103;M...
13292,X,124392181,rs2076165,T,C,426.77,.,0/1,"28,15,0",43,99,None,None,"455,0,826,539,871,1410","13,15,7,8",BaseQRankSum=2.186;DB;DP=43;ExcessHet=3.0103;M...


In [8]:
### 5. Split variants with >1 ALT & Calculate VAF

### 5A. Variants 1 ALT

## PROCESS
# Select variants with 1 ALT
df1 = maf_var_spl[~maf_var_spl['ALT'].str.contains(',')]
# Calculate VAF (& add number of ALT=1)
df1['AD_alt'] = df1['AD'].str.split(',', expand=True)[1].astype(int)
df1['VAF_alt'] = df1['AD'].str.split(',', expand=True)[1].astype(int) / df1['DP'].astype(int)
df1['VAF_ref'] = df1['AD'].str.split(',', expand=True)[0].astype(int) / df1['DP'].astype(int)
df1['ALT_num'] = 1

## CHECK
df1['AD_alt'].value_counts()

<ipython-input-8-ca31d7b2790e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['AD_alt'] = df1['AD'].str.split(',', expand=True)[1].astype(int)
<ipython-input-8-ca31d7b2790e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['VAF_alt'] = df1['AD'].str.split(',', expand=True)[1].astype(int) / df1['DP'].astype(int)
<ipython-input-8-ca31d7b2790e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

2     28
6     21
4     20
3     18
5     18
      ..
81     1
50     1
48     1
47     1
87     1
Name: AD_alt, Length: 63, dtype: int64

In [9]:
### 5B. Variants 2+ ALT

## PROCESS

# Select variants with 2+ ALT & transform df to list
df2 = maf_var_spl[maf_var_spl['ALT'].str.contains(',')]
df2_list = df2.values.tolist()

# Divide variants in 1 line per ALT & calculate VAF
df2_newlist = []
# Loop through all variants
for row in df2_list:
    # Extract info of ALTs and VAF
    ALTs = row[4].split(',')
    ADs = row[8].split(',')
    DP = row[9]
    # Loop to create 1 line per ALT
    for i in range(0, len(ALTs)):
        # Eliminate variants with AD=0 (NOT done)
        # if int(ADs[i+1]) > 1:
            newrow = copy.deepcopy(row)
            # Take ALT and substitute column        
            newrow[4] = ALTs[i]
            # Take AD corresponding ALT in new column
            newrow.append([])
            newrow[16] = int(ADs[i+1])
            # Calculate VAF from corresponding ALT in new column
            newrow.append([])
            newrow[17] = int(ADs[i+1]) / int(DP)
            # Calculate VAF from REF in new column
            newrow.append([])
            newrow[18] = int(ADs[0]) / int(DP)
            # Annotate number of total ALT in new column
            newrow.append([])
            newrow[19] = len(ALTs)
            # Append variant to new list
            df2_newlist.append(newrow)
        
# Transform to dataframe and change column names
df2_newdf = pd.DataFrame(df2_newlist)
df2_newdf.columns = df1.columns

## CHECK
df2_newdf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,GT,AD,DP,GQ,PGT,PID,PL,SB,INFO,AD_alt,VAF_alt,VAF_ref,ALT_num
0,2,99707699,.,CT,C,0.01,.,0/2,"9,2,2,0",13,12,None,None,"12,13,230,0,171,213,46,229,214,265","0,9,0,4",BaseQRankSum=0.306;DP=17;ExcessHet=3.0103;MLEA...,2,0.153846,0.692308,2
1,2,99707699,.,CT,CTT,0.01,.,0/2,"9,2,2,0",13,12,None,None,"12,13,230,0,171,213,46,229,214,265","0,9,0,4",BaseQRankSum=0.306;DP=17;ExcessHet=3.0103;MLEA...,2,0.153846,0.692308,2
2,2,197400449,rs770725816,T,A,308.73,.,0/1,"15,16,0,0",31,99,None,None,"346,0,578,434,448,997,433,554,961,1013","3,12,1,15",BaseQRankSum=-0.262;DB;DP=39;ExcessHet=3.0103;...,16,0.516129,0.483871,2
3,2,197400449,rs770725816,T,TA,308.73,.,0/1,"15,16,0,0",31,99,None,None,"346,0,578,434,448,997,433,554,961,1013","3,12,1,15",BaseQRankSum=-0.262;DB;DP=39;ExcessHet=3.0103;...,0,0.000000,0.483871,2
4,3,70959190,rs112773801,G,GT,3.96,.,0/2,"9,2,2,0",13,3,None,None,"37,3,199,0,185,292,57,226,269,307","9,0,4,0",BaseQRankSum=0.068;DB;DP=16;ExcessHet=3.0103;M...,2,0.153846,0.692308,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,X,124061774,.,T,TAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,1,0.037037,0.925926,4
84,X,124061774,.,T,TTAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,0,0.000000,0.925926,4
85,X,124061774,.,T,TTTAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,1,0.037037,0.925926,4
86,X,124071128,rs759815507,CT,C,3.39,.,0/1,"15,4,4,0",23,20,None,None,"38,0,363,20,261,381,97,362,381,466","13,2,8,0",BaseQRankSum=1.184;DB;DP=32;ExcessHet=3.0103;M...,4,0.173913,0.652174,2


In [10]:
### 5C. Concatenate variants 1ALT and 2+ALT

## PROCESS
maf_var_spl_1alt = pd.concat([df1, df2_newdf], ignore_index=True)

## CHECK
maf_var_spl_1alt

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,GT,AD,DP,GQ,PGT,PID,PL,SB,INFO,AD_alt,VAF_alt,VAF_ref,ALT_num
0,1,1806456,rs77354509,G,A,514.77,.,0/1,"13,14,0",27,99,0|1,1806456_G_A,"543,0,1150,588,1192,1780","12,1,14,0",BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...,14,0.518519,0.481481,1
1,1,1806461,rs17363334,C,T,546.77,.,0/1,"15,15,0",30,99,0|1,1806456_G_A,"575,0,1147,620,1192,1812","12,3,14,1",BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...,15,0.500000,0.500000,1
2,1,43339569,rs16830693,A,G,881.77,.,0/1,"14,27,0",41,99,None,None,"910,0,459,952,541,1493","5,9,6,21",BaseQRankSum=1.575;DB;DP=41;ExcessHet=3.0103;M...,27,0.658537,0.341463,1
3,1,43346404,rs1760670,G,A,232.77,.,0/1,"11,8,0",19,99,None,None,"261,0,437,294,461,755","11,0,8,0",BaseQRankSum=-3.638;DB;DP=19;ExcessHet=3.0103;...,8,0.421053,0.578947,1
4,1,43349193,rs839995,T,C,190.77,.,0/1,"11,6,0",17,99,None,None,"219,0,341,252,360,611","9,2,5,1",BaseQRankSum=3.403;DB;DP=17;ExcessHet=3.0103;M...,6,0.352941,0.647059,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,X,124061774,.,T,TAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,1,0.037037,0.925926,4
459,X,124061774,.,T,TTAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,0,0.000000,0.925926,4
460,X,124061774,.,T,TTTAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,1,0.037037,0.925926,4
461,X,124071128,rs759815507,CT,C,3.39,.,0/1,"15,4,4,0",23,20,None,None,"38,0,363,20,261,381,97,362,381,466","13,2,8,0",BaseQRankSum=1.184;DB;DP=32;ExcessHet=3.0103;M...,4,0.173913,0.652174,2


In [11]:
### 6. Annotate variant type

## PROCESS

# Define function
def variant_type(var):
    if (len(var[3]) == len(var[4])) & (len(var[3]) == 1):
        return 'SNV'
    elif len(var[3]) != len(var[4]):
        return 'Indel'
    elif (len(var[3]) == len(var[4])) & (len(var[3]) > 1):
        return 'MNV'
    else:
        return 'Unknown'

# Add type of variant
maf_var_spl_1alt['var_type'] = maf_var_spl_1alt.apply(lambda x: variant_type(x), axis=1)

## CHECK
print(maf_var_spl_1alt['var_type'].value_counts())
maf_var_spl_1alt


SNV      311
Indel    152
Name: var_type, dtype: int64


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,GT,AD,DP,GQ,PGT,PID,PL,SB,INFO,AD_alt,VAF_alt,VAF_ref,ALT_num,var_type
0,1,1806456,rs77354509,G,A,514.77,.,0/1,"13,14,0",27,99,0|1,1806456_G_A,"543,0,1150,588,1192,1780","12,1,14,0",BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...,14,0.518519,0.481481,1,SNV
1,1,1806461,rs17363334,C,T,546.77,.,0/1,"15,15,0",30,99,0|1,1806456_G_A,"575,0,1147,620,1192,1812","12,3,14,1",BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...,15,0.500000,0.500000,1,SNV
2,1,43339569,rs16830693,A,G,881.77,.,0/1,"14,27,0",41,99,None,None,"910,0,459,952,541,1493","5,9,6,21",BaseQRankSum=1.575;DB;DP=41;ExcessHet=3.0103;M...,27,0.658537,0.341463,1,SNV
3,1,43346404,rs1760670,G,A,232.77,.,0/1,"11,8,0",19,99,None,None,"261,0,437,294,461,755","11,0,8,0",BaseQRankSum=-3.638;DB;DP=19;ExcessHet=3.0103;...,8,0.421053,0.578947,1,SNV
4,1,43349193,rs839995,T,C,190.77,.,0/1,"11,6,0",17,99,None,None,"219,0,341,252,360,611","9,2,5,1",BaseQRankSum=3.403;DB;DP=17;ExcessHet=3.0103;M...,6,0.352941,0.647059,1,SNV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,X,124061774,.,T,TAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,1,0.037037,0.925926,4,Indel
459,X,124061774,.,T,TTAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,0,0.000000,0.925926,4,Indel
460,X,124061774,.,T,TTTAAAAAA,0,.,0/0,"25,0,1,0,1,0",27,42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...,1,0.037037,0.925926,4,Indel
461,X,124071128,rs759815507,CT,C,3.39,.,0/1,"15,4,4,0",23,20,None,None,"38,0,363,20,261,381,97,362,381,466","13,2,8,0",BaseQRankSum=1.184;DB;DP=32;ExcessHet=3.0103;M...,4,0.173913,0.652174,2,Indel


In [12]:
### 7. Reorder columns and create new column for VEP output

## PROCESS
maf_var_spl_1alt["VEP"] = ""
maf_var_spl_1alt = maf_var_spl_1alt[['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'VEP',
                                     'AD_alt', 'DP', 'VAF_alt', 'VAF_ref', 'ALT_num', 'var_type',
                                     'GT', 'AD', 'GQ', 'PGT', 'PID', 'PL', 'SB', 'INFO']]
## CHECK
maf_var_spl_1alt

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,VEP,AD_alt,DP,VAF_alt,VAF_ref,ALT_num,var_type,GT,AD,GQ,PGT,PID,PL,SB,INFO
0,1,1806456,rs77354509,G,A,514.77,.,,14,27,0.518519,0.481481,1,SNV,0/1,"13,14,0",99,0|1,1806456_G_A,"543,0,1150,588,1192,1780","12,1,14,0",BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...
1,1,1806461,rs17363334,C,T,546.77,.,,15,30,0.500000,0.500000,1,SNV,0/1,"15,15,0",99,0|1,1806456_G_A,"575,0,1147,620,1192,1812","12,3,14,1",BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...
2,1,43339569,rs16830693,A,G,881.77,.,,27,41,0.658537,0.341463,1,SNV,0/1,"14,27,0",99,None,None,"910,0,459,952,541,1493","5,9,6,21",BaseQRankSum=1.575;DB;DP=41;ExcessHet=3.0103;M...
3,1,43346404,rs1760670,G,A,232.77,.,,8,19,0.421053,0.578947,1,SNV,0/1,"11,8,0",99,None,None,"261,0,437,294,461,755","11,0,8,0",BaseQRankSum=-3.638;DB;DP=19;ExcessHet=3.0103;...
4,1,43349193,rs839995,T,C,190.77,.,,6,17,0.352941,0.647059,1,SNV,0/1,"11,6,0",99,None,None,"219,0,341,252,360,611","9,2,5,1",BaseQRankSum=3.403;DB;DP=17;ExcessHet=3.0103;M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,X,124061774,.,T,TAAAAAA,0,.,,1,27,0.037037,0.925926,4,Indel,0/0,"25,0,1,0,1,0",42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...
459,X,124061774,.,T,TTAAAAAA,0,.,,0,27,0.000000,0.925926,4,Indel,0/0,"25,0,1,0,1,0",42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...
460,X,124061774,.,T,TTTAAAAAA,0,.,,1,27,0.037037,0.925926,4,Indel,0/0,"25,0,1,0,1,0",42,None,None,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...
461,X,124071128,rs759815507,CT,C,3.39,.,,4,23,0.173913,0.652174,2,Indel,0/1,"15,4,4,0",20,None,None,"38,0,363,20,261,381,97,362,381,466","13,2,8,0",BaseQRankSum=1.184;DB;DP=32;ExcessHet=3.0103;M...


### 8. VEP ANNOTATION

singularity exec vep.simg vep -i /workspace/projects/clonalhemato_ukb/analysis_50k_202103/analysis_ch_genes_vep_202104/results/ch_variants_1allel_maf1.txt -o /workspace/projects/clonalhemato_ukb/analysis_50k_202103/analysis_ch_genes_vep_202104/veptest/veptestmaf1_20april_VCF_gnomAD.txt --assembly GRCh38 --no_stats --cache --offline --symbol --protein --vcf --canonical --af_gnomad --dir /workspace/datasets/vep

In [13]:
### CHECK

from io import StringIO 

filename = '/workspace/projects/clonalhemato_ukb/analysis_50k_202103/analysis_ch_genes_vep_202104/veptest/veptestmaf1_21april_VCF_gnomAD.txt'

lines = ''.join([line for line in open(filename) if not line.startswith("##")])
vep_test_vcf_gnomad = pd.read_csv(StringIO(lines), sep= '\t')


vep_test_vcf_gnomad

#print(vep_test_vcf_gnomad['VEP'][0].split(',', expand=True))

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,VEP,AD_alt,DP,VAF_alt,VAF_ref,ALT_num,var_type,GT,AD,GQ,PGT,PID,PL,SB,INFO
0,1,1806456,rs77354509,G,A,514.77,.,CSQ=A|intron_variant|MODIFIER|GNB1|ENSG0000007...,14,27,0.518519,0.481481,1,SNV,0/1,"13,14,0",99,0|1,1806456_G_A,"543,0,1150,588,1192,1780","12,1,14,0",BaseQRankSum=-3.829;DB;DP=28;ExcessHet=3.0103;...
1,1,1806461,rs17363334,C,T,546.77,.,CSQ=T|intron_variant|MODIFIER|GNB1|ENSG0000007...,15,30,0.500000,0.500000,1,SNV,0/1,"15,15,0",99,0|1,1806456_G_A,"575,0,1147,620,1192,1812","12,3,14,1",BaseQRankSum=-4.741;DB;DP=31;ExcessHet=3.0103;...
2,1,43339569,rs16830693,A,G,881.77,.,CSQ=G|splice_region_variant&synonymous_variant...,27,41,0.658537,0.341463,1,SNV,0/1,"14,27,0",99,NaN,NaN,"910,0,459,952,541,1493","5,9,6,21",BaseQRankSum=1.575;DB;DP=41;ExcessHet=3.0103;M...
3,1,43346404,rs1760670,G,A,232.77,.,CSQ=A|intron_variant|MODIFIER|MPL|ENSG00000117...,8,19,0.421053,0.578947,1,SNV,0/1,"11,8,0",99,NaN,NaN,"261,0,437,294,461,755","11,0,8,0",BaseQRankSum=-3.638;DB;DP=19;ExcessHet=3.0103;...
4,1,43349193,rs839995,T,C,190.77,.,CSQ=C|intron_variant|MODIFIER|MPL|ENSG00000117...,6,17,0.352941,0.647059,1,SNV,0/1,"11,6,0",99,NaN,NaN,"219,0,341,252,360,611","9,2,5,1",BaseQRankSum=3.403;DB;DP=17;ExcessHet=3.0103;M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,X,124061774,.,T,TAAAAAA,0.00,.,CSQ=AAAAAA|inframe_insertion|MODERATE|STAG2|EN...,1,27,0.037037,0.925926,4,Indel,0/0,"25,0,1,0,1,0",42,NaN,NaN,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...
459,X,124061774,.,T,TTAAAAAA,0.00,.,CSQ=TAAAAAA|frameshift_variant|HIGH|STAG2|ENSG...,0,27,0.000000,0.925926,4,Indel,0/0,"25,0,1,0,1,0",42,NaN,NaN,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...
460,X,124061774,.,T,TTTAAAAAA,0.00,.,CSQ=TTAAAAAA|stop_gained&frameshift_variant|HI...,1,27,0.037037,0.925926,4,Indel,0/0,"25,0,1,0,1,0",42,NaN,NaN,"0,79,1149,52,1112,1120,59,1113,1110,1122,42,10...","16,9,2,0",BaseQRankSum=1.369;DP=28;ExcessHet=3.0103;MLEA...
461,X,124071128,rs759815507,CT,C,3.39,.,CSQ=-|intron_variant|MODIFIER|STAG2|ENSG000001...,4,23,0.173913,0.652174,2,Indel,0/1,"15,4,4,0",20,NaN,NaN,"38,0,363,20,261,381,97,362,381,466","13,2,8,0",BaseQRankSum=1.184;DB;DP=32;ExcessHet=3.0103;M...
